In [1]:
import pandas as pd
from streamlit_assets.components import get_recos_users
from sklearn.preprocessing import LabelEncoder
from surprise import SVD, NMF, KNNBasic, SVDpp
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader
import numpy as np

### Exploring metacritic data for recos

In [11]:
meta_df = pd.read_csv('data/metacritic.csv').dropna(subset='critic_meta_score')
#meta_df = meta_df.rename(columns={"title":"item_id"})

meta_df.head()


,title,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,user_score,release_date,link,critics_reviews_link
0,Rectify: Season 4,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,"Oct 27, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
1,Rectify: Season 4,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,"Oct 25, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
2,Rectify: Season 4,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
3,Rectify: Season 4,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...
4,Rectify: Season 4,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...


In [20]:
meta_df['title_group'] = meta_df['title'].str.split(': Season').str[0]
meta_df

,title,rank,critic_meta_score,review_content,review_source,author,review_date,summary,meta_score,user_score,release_date,link,critics_reviews_link,title_group
0,Rectify: Season 4,1,100.0,It allows us to know and care for these charac...,Collider,Allison Keene,"Oct 27, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify
1,Rectify: Season 4,1,100.0,"Rectify, a drama entering its final season on ...",The New York Times,James Poniewozik,"Oct 25, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify
2,Rectify: Season 4,1,100.0,No other series so poignantly probes the human...,Salon,Melanie McFarland,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify
3,Rectify: Season 4,1,100.0,None of these characters is particularly happy...,Yahoo TV,Ken Tucker,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify
4,Rectify: Season 4,1,100.0,Rectify is the best series I have ever seen on...,The Daily Beast,Malcolm Jones,"Oct 26, 2016",Daniel tries to start a new life outside of Pa...,99,8.7,"October 26, 2016",https://www.metacritic.com/tv/rectify/season-4,https://www.metacritic.com/tv/rectify/season-4...,Rectify
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52889,The 1/2 Hour News Hour: Season 1,3139,0.0,You couldn't skewer a cube of tofu with materi...,Slate,Troy Patterson,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,"February 18, 2007",https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...,The 1/2 Hour News Hour
52890,The 1/2 Hour News Hour: Season 1,3139,0.0,"It's hard not to keep staring perplexed, squin...",The New Republic,Sacha Zimmerman,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,"February 18, 2007",https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...,The 1/2 Hour News Hour
52891,The 1/2 Hour News Hour: Season 1,3139,0.0,It's even worse than anyone imagined.,The A.V. Club,Amelie Gillette,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,"February 18, 2007",https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...,The 1/2 Hour News Hour
52892,The 1/2 Hour News Hour: Season 1,3139,10.0,The humor is so predictable and so stale that ...,Chicago Tribune,Maureen Ryan,NaN,"The right-wing answer to ""The Daily Show With ...",13,2.8,"February 18, 2007",https://www.metacritic.com/tv/the-12-hour-news...,https://www.metacritic.com/tv/the-12-hour-news...,The 1/2 Hour News Hour


In [26]:
meta_df_grouped = meta_df.groupby('title_group').mean()
meta_df_grouped

,rank,critic_meta_score,meta_score
title_group,,,
#blackAF,2019.0,56.111111,61.0
'Til Death Do Us Part,2892.0,42.750000,43.0
1 vs 100,2437.0,53.636364,54.0
10 Items Or Less,2917.0,41.769231,42.0
10 Things I Hate About You,1608.0,66.111111,66.0
...,...,...,...
jeen-yuhs: A Kanye Trilogy,1201.0,71.875000,71.0
mixed-ish,1226.0,71.666667,70.0
neXt (2020),2179.0,59.666667,58.0


In [27]:
id_maker = LabelEncoder()
meta_df['user_id'] = id_maker.fit_transform(meta_df['author'])
meta_df['item_id'] = meta_df['title']
meta_df['rating'] = meta_df['critic_meta_score'].apply(int)

reviews = meta_df[['user_id','item_id','rating']]

In [28]:
tv_list = ['Breaking Bad: Season 5',
           'Rectify: Season 4',
           'Better Call Saul: Season 5',
           'The Leftovers: Season 1',
           'Lodge 49',
           'The Underground Railroad',
           'Master of None: Season 1',
           'Normal People: Season 1'
          ]


In [29]:
df_recos = get_recos_users(tv_list,reviews, filter_items=True,filter_n=20, default_rating=90)

C:\git\reco_system\streamlit_assets\components.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_reviews = df.append(new_df).reset_index(drop=True)


In [33]:
df_merged = meta_df.merge(df_recos, on='item_id').drop_duplicates(subset='item_id').sort_values(by='user match', ascending=False).reset_index(drop=True)
df_merged[['title','summary','user match','meta_score']].head(50)

,title,summary,user match,meta_score
0,The Handmaid's Tale: Season 1,"In the totalitarian society of Gilead, a handm...",94.739800,92
1,The Sopranos: Season 3,"Meet Tony Soprano: your average, middle-aged b...",92.389561,97
2,Master of None: Season 2,Dev (Aziz Ansari) travels to Italy in the seco...,92.146731,91
3,It's a Sin,The five-part miniseries written and created b...,92.130487,91
4,Fleabag: Season 2,A priest (Andrew Scott) helps Fleabag change t...,91.829227,96
5,Game of Thrones: Season 2,The Seven Kingdoms are at war with three kings...,91.648154,90
6,Orange is the New Black: Season 2,Season two finds Piper (Taylor Schilling) spen...,91.451834,89
7,Deadwood: The Movie,"Picking up 10 years after the end of season 3,...",91.438298,86
8,Game of Thrones: Season 4,The fourth season finds King Joffrey preparing...,90.859334,94
9,The Night Of: Season 1,The limited series created by Steven Zaillian ...,90.742563,90


In [34]:
df_merged['title_group'] = df_merged['title'].str.split(': Season').str[0]

In [35]:
df_grouped = df_merged.groupby('title_group').mean().sort_values(by='user match', ascending = False)

In [37]:
df_grouped.head(50)

,rank,critic_meta_score,meta_score,user_id,rating,user match
title_group,,,,,,
Master of None,80.000000,100.000000,91.000000,1473.000000,100.000000,92.146731
It's a Sin,77.000000,70.000000,91.000000,775.000000,70.000000,92.130487
Fleabag,25.000000,94.000000,96.000000,73.000000,94.000000,91.829227
Deadwood: The Movie,209.000000,100.000000,86.000000,968.000000,100.000000,91.438298
The Night Of,94.000000,90.000000,90.000000,431.000000,90.000000,90.742563
Treme,158.000000,100.000000,88.000000,803.000000,100.000000,90.601665
The Beatles: Get Back,234.000000,80.000000,85.000000,655.000000,80.000000,90.101681
The Shield,54.000000,100.000000,92.000000,1074.000000,100.000000,90.005386
The White Lotus,380.000000,90.000000,82.000000,606.000000,90.000000,89.543911
